In [58]:
from mexc_sdk import Spot
import os 
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from datetime import datetime

In [59]:
now = datetime.now().date()

In [60]:
symbol = input("Symbol = ")
folder_save = f"../../data/{symbol}"

is_quick_get = input("is quick get? (y/n)").strip().lower() == "y"
if is_quick_get:
    start_date = f"{now.year}-01-01"
else:
    start_date = input("start date? (YYYY-MM-DD)").strip().lower()

to_date = str(now)

is_get_now = input("Get now? (y/n)").strip().lower()[0] == "y"
concanate_with_old = input("concanate with old data? (y/n)").strip().lower()[0] == "y"

interval = input("""interval = 
                 
                "1m" - "1 minute"
                "5m" - "5 minute"
                "15m" - "15 minute"
                "30m" - "30 minute"
                "60m" - "60 minute"
                "4h" - "4 hour"
                "1d" - "1 day"
                "1W" - "1 week"
                "1M" - "1 month"
                 """).strip()

In [61]:
load_dotenv(dotenv_path="./apikey.txt")

True

In [62]:
access_key = os.getenv("access_key")
secret_key = os.getenv("secret_key")

In [63]:
spot = Spot(api_key=access_key, api_secret=secret_key)

In [64]:
data_from_mexc = spot.klines(symbol.upper() + "USDT", interval=interval, 
    # options={
    #     "startTime": start_date,
    #     "endTime": to_date
    # }
    )

In [65]:
df_origin_mexc = pd.DataFrame(data_from_mexc)

In [66]:
df_origin_mexc.columns

RangeIndex(start=0, stop=8, step=1)

In [67]:
df_origin_mexc["Date"] = pd.to_datetime(df_origin_mexc.iloc[:, 0], unit="ms")

In [68]:
mapping_prices = {1: "Open",
                  2: "High",
                  3: "Low",
                  4: "Close",
                  5: "Volume", }

In [69]:
for index in mapping_prices:
    df_origin_mexc[mapping_prices[index]] = df_origin_mexc.iloc[:, index]

In [70]:
df_from_mexc = df_origin_mexc[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]

In [71]:
now = datetime.now().date()

In [72]:
now

datetime.date(2024, 12, 1)

In [73]:
df_from_mexc

,Open,High,Low,Close,Volume,Date
0,1891.59,1905.6,1884.64,1891.73,33691.2519,2023-07-21
1,1891.73,1897.34,1852.27,1866.52,19609.70144,2023-07-22
2,1866.52,1905,1858.4,1888.74,26057.72458,2023-07-23
3,1888.74,1890.86,1834.46,1850,39554.46865,2023-07-24
4,1850,1868.42,1845.45,1857.66,32669.27774,2023-07-25
...,...,...,...,...,...,...
495,3324.74,3684.84,3302.66,3653.29,191802.24952,2024-11-27
496,3653.29,3661.72,3529.76,3578.33,157640.01591,2024-11-28
497,3578.33,3645.67,3534.97,3592.22,172687.94079,2024-11-29
498,3592.22,3738.98,3569,3703.59,189472.0092,2024-11-30


In [74]:
if not is_get_now:
    df_from_mexc = df_from_mexc[df_from_mexc["Date"] != str(now)]

In [75]:
if concanate_with_old:
    old_df = pd.read_csv(f"{folder_save}/{symbol.lower()}_to_now_{interval.lower()}.csv")
    old_df["Date"] = pd.to_datetime(old_df["Date"])
    old_df = old_df[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]
    max_date_in_old_df = str(old_df["Date"].max())
    df_today = df_from_mexc[df_from_mexc["Date"] > max_date_in_old_df]
else:
    df_today = df_from_mexc


In [76]:
df_today["Date"] = df_today["Date"].dt.date
if concanate_with_old:
    old_df["Date"] = old_df["Date"].dt.date
    df = pd.concat([old_df, df_today], ignore_index=True)
else:
    df = df_today
df["Date"] = pd.to_datetime(df["Date"])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26596\1533491661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_today["Date"] = df_today["Date"].dt.date
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26596\1533491661.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"])


In [77]:
df

,Open,High,Low,Close,Volume,Date
0,1891.59,1905.6,1884.64,1891.73,33691.2519,2023-07-21
1,1891.73,1897.34,1852.27,1866.52,19609.70144,2023-07-22
2,1866.52,1905,1858.4,1888.74,26057.72458,2023-07-23
3,1888.74,1890.86,1834.46,1850,39554.46865,2023-07-24
4,1850,1868.42,1845.45,1857.66,32669.27774,2023-07-25
...,...,...,...,...,...,...
494,3415.08,3462.34,3253,3324.74,200489.11075,2024-11-26
495,3324.74,3684.84,3302.66,3653.29,191802.24952,2024-11-27
496,3653.29,3661.72,3529.76,3578.33,157640.01591,2024-11-28
497,3578.33,3645.67,3534.97,3592.22,172687.94079,2024-11-29


In [78]:
symbol = symbol.replace(".", "_").lower()

In [79]:
min_date = df["Date"].min().date()
max_date = df["Date"].max().date()

os.makedirs(folder_save, exist_ok=True)
file_name = f"{folder_save}/{symbol}_to_now_{interval.lower()}.csv"

In [80]:
file_name = file_name.replace("-", "_")

In [81]:
file_name

'../../data/eth/eth_to_now_1d.csv'

In [82]:
df.to_csv(file_name, index=False)